# creo un ruleset "ground truth" con 3 regole

In [ ]:
from bpllib.rules._discrete_constraint import DiscreteConstraint
from bpllib.rules._rule import Rule
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# random rules
r1 = Rule(constraints={0: DiscreteConstraint(value=0, index=0),
                       1: DiscreteConstraint(value=1, index=1),
                       3: DiscreteConstraint(value=0, index=3)})

r2 = Rule(constraints={2: DiscreteConstraint(value=1, index=2),
                       4: DiscreteConstraint(value=1, index=4)})

#r3 = Rule(constraints={5: DiscreteConstraint(value=1, index=5),
#                       6: DiscreteConstraint(value=0, index=6)})

rule_set = [r1, r2]  #, r3]

In [2]:
# random data

X_train = np.random.randint(0, 2, size=(100, 20))
y_train = np.array([any([r.covers(x) for r in rule_set]) for x in X_train])
X_test = np.random.randint(0, 2, size=(100, 20))
y_test = np.array([any([r.covers(x) for r in rule_set]) for x in X_test])

In [3]:
y_train.sum()

39

In [4]:
import itertools

def generate_rules(m, n_features=7):
    all_features = list(range(n_features))
    combos = itertools.combinations(all_features, m)
    rules = set()
    for combo in combos:
        for combo_values in itertools.product([0,1], repeat=m):
            rule = Rule(constraints={})
            for i in range(m):
                if combo_values[i] == 0:
                    rule.constraints[combo[i]] = DiscreteConstraint(value=0, index=combo[i])
                else:
                    rule.constraints[combo[i]] = DiscreteConstraint(value=1, index=combo[i])
            rules.add(rule)
    return rules

In [5]:
rules_space = set.union(*[generate_rules(m=m, n_features=5) for m in range(1, 5)])
print(len(rules_space))

210


In [6]:
# nonsense
VS_rules = {r for r in rules_space if not any([r.covers(x) for x in X_train[y_train==0]]) and any([r.covers(x) for x in X_train[y_train==1]])}

In [7]:
# sample from rules_space
#T = 100
#uniform_sample = np.random.choice(list(rules_space), size=T, replace=True)


In [8]:
#classification = np.array([sum([r.covers(x) for r in uniform_sample]) >= T // 2 for x in X_train])

In [9]:
#classification.sum()

In [6]:
# Generate the set of all possible tuples of up to k elements each, chosen from rules_space
def generate_tuples(k):
    all_tuples = []
    for i in range(1, k+1):
        print(len(all_tuples))
        tuples = list(itertools.combinations(rules_space, i))
        all_tuples = all_tuples + tuples
    return all_tuples

In [ ]:
x = generate_tuples(4)  # one minute...

In [ ]:
len(x)

In [13]:
print(x[-1])

(X[0] == 1 ^ X[1] == 1 ^ X[3] == 1 ^ X[5] == 1, X[0] == 0 ^ X[1] == 1 ^ X[2] == 0 ^ X[5] == 1, X[3] == 1 ^ X[4] == 0)


In [ ]:
hypothesis_space = x

In [ ]:
len(hypothesis_space)


In [ ]:
# pick one element
h = next(iter(hypothesis_space))

In [ ]:
print(h)

In [ ]:
len(X_train)

In [25]:
#from tqdm import tqdm
#VS2 = {h for h in tqdm(hypothesis_space) if all([any([r.covers(x) for r in h]) for x in X_train[y_train==1]]) and not any([any([r.covers(x) for r in h]) for x in X_train[y_train==0]])}

  2%|▏         | 204286/9002007 [00:07<05:14, 27984.83it/s]

KeyboardInterrupt: 

In [ ]:
VS = set()
from tqdm import tqdm
for h in tqdm(hypothesis_space):
    ok = True
    for x in X_train[y_train==1]:
        if not any([r.covers(x) for r in h]):
            ok = False
            break
    for x in X_train[y_train==0]:
        if any([r.covers(x) for r in h]):
            ok = False
            break
    if ok:
        VS.add(h)


In [60]:
len(VS)

1

In [61]:
VS

{(X[2] == 1 ^ X[4] == 1, X[0] == 0 ^ X[1] == 1 ^ X[3] == 0)}

In [31]:
# sample from VS
T = 100
uniform_sample = np.random.choice(list(VS), size=T, replace=True)

/var/folders/zg/z65vx93j21vb3xf7sqmrn2n00000gn/T/ipykernel_33981/596547705.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  uniform_sample = np.random.choice(list(VS), size=T, replace=True)


In [32]:
y_pred = np.array([sum([any([r.covers(x) for r in h]) for h in uniform_sample]) >= T // 2 for x in X_test])

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

1.0

In [36]:
beta = 1
p = np.array([np.exp(-beta*len(h)) for h in VS])
p = p / sum(p)
weighted_sample = np.random.choice(list(VS), size=T, replace=True, p=p)

/var/folders/zg/z65vx93j21vb3xf7sqmrn2n00000gn/T/ipykernel_33981/1819034514.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  weighted_sample = np.random.choice(list(VS), size=T, replace=True, p=p)


In [37]:
y_pred = np.array([sum([any([r.covers(x) for r in h]) for h in uniform_sample]) >= T // 2 for x in X_test])

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

1.0